## Network Traffic Dataset for malicious attack

This dataset of network traffic flow is generated by CICFlowMeter, indicate whether the traffic is malicious attack (Bot) or not (Benign).                             
CICFlowMeter - network traffic flow generator generates 69 statistical features such as Duration, Number of packets, Number of bytes, Length of packets, etc are also calculated separately in the forward and reverse direction.   
The output of the application is the CSV file format with two columns labeled for each flow, namely Benign or Bot.
The dataset has been organized per day, for each day the raw data including the network traffic (Pcaps) and event logs (windows and Ubuntu event Logs) per machine
are recorded.                  Download the dataset from the below wget command line provided.

In [ ]:
! wget https://cse-cic-ids2018.s3.ca-central-1.amazonaws.com/Processed+Traffic+Data+for+ML+Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv

## Install Libraries

In [1]:
! pip install imblearn
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tempfile
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import KFold
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

## Declare Variables

In [3]:
INPUT_FEATURE = 'x'

lstZerodrp = ['Timestamp', 'BwdPSHFlags', 'FwdURGFlags', 'BwdURGFlags', 'CWEFlagCount', 'FwdBytsbAvg', 'FwdPktsbAvg',
              'FwdBlkRateAvg', 'BwdBytsbAvg',
              'BwdBlkRateAvg', 'BwdPktsbAvg']

lstScaledrp = ['FwdPSHFlags', 'FINFlagCnt', 'SYNFlagCnt', 'RSTFlagCnt', 'PSHFlagCnt', 'ACKFlagCnt', 'URGFlagCnt',
               'ECEFlagCnt']

DATA_FILE = 'Network_Traffic.csv'

In [4]:
def read_dataFile():
    """
    Reads data file and returns dataframe result
    """
    chunksize = 10000
    chunk_list = []
    missing_values = ["n/a", "na", "--", "Infinity", "infinity", "Nan", "NaN"]

    for chunk in pd.read_csv(DATA_FILE, chunksize=chunksize, na_values=missing_values):
        chunk_list.append(chunk)
        break
    dataFrme = pd.concat(chunk_list)

    lstcols = []
    for i in dataFrme.columns:
        i = str(i).replace(' ', '').replace('/', '')
        lstcols.append(i)
    dataFrme.columns = lstcols
    dfAllCpy = dataFrme.copy()
    dataFrme = dataFrme.drop(lstZerodrp, axis=1)
    return dataFrme

## Network Traffic Input Dataset 

### Attribute Information
    Features extracted from the captured traffic using CICFlowMeter-V3 = 69
    After removal of noise/unwarranted features, number of feature columns chosen: 10
    Features: FlowDuration,BwdPktLenMax,FlowIATStd,FwdPSHFlags,BwdPktLenMean,FlowIATMean,BwdIATMean,
              FwdSegSizeMin,InitBwdWinByts,BwdPktLenMin
    Flows labelled: Bot or Benign

In [5]:
read_dataFile().head()

,DstPort,Protocol,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMax,FwdPktLenMin,FwdPktLenMean,...,FwdSegSizeMin,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,443,6,141385,9,7,553,3773,202,0,61.444444,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,281,2,1,38,0,38,0,19.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,279824,11,15,1086,10527,385,0,98.727273,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,132,2,0,0,0,0,0,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,274016,9,13,1285,6141,517,0,142.777778,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [6]:
def preprocess_na(dataFrme):
    """
    Removing NA values
    """
    na_lst = dataFrme.columns[dataFrme.isna().any()].tolist()
    for j in na_lst:
        dataFrme[j].fillna(0, inplace=True)
    return dataFrme

In [7]:
def create_features_label(dataFrme):
    """
    Create independent and Dependent Features
    """
    columns = dataFrme.columns.tolist()
    # Filter the columns to remove data we do not want 
    columns = [c for c in columns if c not in ["Label"]]
    # Store the variable we are predicting 
    target = "Label"
    # Define a random state 
    state = np.random.RandomState(42)
    X = dataFrme[columns]
    Y = dataFrme[target]
    return X, Y

In [8]:
def label_substitution(dataFrme):
    """
    Label substitution : 'Benign'as 0, 'Bot'as 1
    """
    dictLabel = {'Benign': 0, 'Bot': 1}
    dataFrme['Label'] = dataFrme['Label'].map(dictLabel)

    LABELS = ['Benign', 'Bot']
    count_classes = pd.value_counts(dataFrme['Label'], sort=True)
    
    # Get the Benign and the Bot values 
    Benign = dataFrme[dataFrme['Label'] == 0]
    Bot = dataFrme[dataFrme['Label'] == 1]
    return dataFrme

In [9]:
def handle_class_imbalance(X,Y):
    """
    Handle Class imbalancement 
    """
#    os_us = SMOTETomek(ratio=0.5)
#    X_res, y_res = os_us.fit_sample(X, Y)
    ros = RandomOverSampler(random_state=50)
    X_res, y_res = ros.fit_sample(X, Y)
    ibtrain_X = pd.DataFrame(X_res,columns=X.columns)
    ibtrain_y = pd.DataFrame(y_res,columns=['Label']) 
    return ibtrain_X,ibtrain_y

In [10]:
def correlation_features(ibtrain_X):
    """
    Feature Selection - Correlation Ananlysis 
    """
    corr = ibtrain_X.corr()
    cor_columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= 0.9:
                if cor_columns[j]:
                    cor_columns[j] = False

    dfcorr_features = ibtrain_X[corr.columns[cor_columns]]
    return dfcorr_features

In [11]:
def top_ten_features(dfcorr_features, ibtrain_X, ibtrain_y):
    """
    Feature Selection - SelectKBest : Return best 10 features. 
    """
    feat_X = dfcorr_features
    feat_y = ibtrain_y['Label']
    # apply SelectKBest class to extract top 10 best features
    bestfeatures = SelectKBest(score_func=f_classif, k=10)
    fit = bestfeatures.fit(feat_X, feat_y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(feat_X.columns)
    # concat two dataframes for better visualization
    featureScores = pd.concat([dfcolumns, dfscores], axis=1)
    featureScores.columns = ['Features', 'Score']  # naming the dataframe columns
    final_feature = featureScores.nlargest(10, 'Score')['Features'].tolist()
    dictLabel1 = {'Benign': 0, 'Bot': 1}
    ibtrain_y['Label'] = ibtrain_y['Label'].map(dictLabel1)
    selected_X = ibtrain_X[final_feature]
    selected_Y = ibtrain_y['Label']
    return selected_X, selected_Y, final_feature

In [12]:
def normalize_data(selected_X, selected_Y):
    """
    Normalize data 
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    selected_X = pd.DataFrame(scaler.fit_transform(selected_X), columns=selected_X.columns, index=selected_X.index)
    trainX, testX, trainY, testY = train_test_split(selected_X, selected_Y, test_size=0.25)
    print('-----------------------------------------------------------------')
    print("## Final features and Data pre-process for prediction")
    print('-----------------------------------------------------------------')
    print(testX)
    return trainX, testX, trainY, testY

## Definition of Serving Input Receiver Function

In [13]:
def serving_input_receiver_fn():
    """
    This is used to define inputs to serve the model.
    :return: ServingInputReciever
    """
    receiver_tensors = {
        'FlowDuration': tf.placeholder(tf.float32, [None, 1]),
        'BwdPktLenMax': tf.placeholder(tf.float32, [None, 1]),
        'FlowIATStd': tf.placeholder(tf.float32, [None, 1]),
        'BwdPktLenMean': tf.placeholder(tf.float32, [None, 1]),
        'FwdPSHFlags': tf.placeholder(tf.float32, [None, 1]),
        'FlowIATMean': tf.placeholder(tf.float32, [None, 1]),
        'BwdIATMean': tf.placeholder(tf.float32, [None, 1]),
        'FwdSegSizeMin': tf.placeholder(tf.float32, [None, 1]),
        'InitBwdWinByts': tf.placeholder(tf.float32, [None, 1]),
        'BwdPktLenMin': tf.placeholder(tf.float32, [None, 1])

    }

    # Convert give inputs to adjust to the model.
    features = {
        INPUT_FEATURE: tf.concat([
            receiver_tensors['FlowDuration'],
            receiver_tensors['BwdPktLenMax'],
            receiver_tensors['FlowIATStd'],
            receiver_tensors['BwdPktLenMean'],
            receiver_tensors['FwdPSHFlags'],
            receiver_tensors['FlowIATMean'],
            receiver_tensors['BwdIATMean'],
            receiver_tensors['FwdSegSizeMin'],
            receiver_tensors['InitBwdWinByts'],
            receiver_tensors['BwdPktLenMin'],
        ], axis=1)
    }
    return tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors,
                                                    features=features)

In [14]:
def get_model(trainX, trainY, testX, testY, final_feature):
    
    """
    Training and Evalution
    """
    
    TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
    TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "network/")
    TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "network/")

    train_X = np.asarray(trainX)
    train_y = np.asarray(trainY)
    feature_columns = [tf.feature_column.numeric_column(INPUT_FEATURE, shape=[10])]

    config = tf.estimator.RunConfig(model_dir=TF_MODEL_DIR, save_summary_steps=10, save_checkpoints_steps=10)

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={INPUT_FEATURE: train_X},
        y=train_y,
        batch_size=32,
        num_epochs=10,
        shuffle=False
    )

    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={INPUT_FEATURE: train_X},
        y=train_y,
        batch_size=32,
        num_epochs=10,
        shuffle=True,
        queue_capacity=10,
        num_threads=1
    )

    model = tf.estimator.DNNClassifier(hidden_units=[13, 65, 110],
                                       feature_columns=feature_columns,
                                       model_dir=TF_MODEL_DIR,
                                       n_classes=2, config=config
                                       )

    export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn,
                                        max_steps=1)

    eval_spec = tf.estimator.EvalSpec(input_fn=test_input_fn,
                                      steps=1,
                                      exporters=export_final,
                                      throttle_secs=1,
                                      start_delay_secs=1)

    result = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    print(result)

## Train and Save Network Traffic Model

In [15]:
def main(unused_args):
    
    tf.logging.set_verbosity(tf.logging.INFO)
    
    '''Reads data file and returns dataframe result'''
    dataFrme = read_dataFile()
  
    ''' Removing NA values'''
    dataFrme = preprocess_na(dataFrme)

    '''Create independent and Dependent Features'''
    X, Y = create_features_label(dataFrme)

    '''Label substitution : 'Benign'as 0, 'Bot'as 1'''
    dataFrme = label_substitution(dataFrme)

    '''Handle Class imbalancement'''
    ibtrain_X, ibtrain_y = handle_class_imbalance(X, Y)

    '''Feature Selection - Correlation Ananlysis'''
    dfcorr_features = correlation_features(ibtrain_X)

    '''Feature Selection - SelectKBest : Return best 10 features'''
    selected_X, selected_Y, final_feature = top_ten_features(dfcorr_features, ibtrain_X, ibtrain_y)

    '''Normalize data '''
    trainX, testX, trainY, testY = normalize_data(selected_X, selected_Y)
    
    '''Train and Evaluate'''
    get_model(trainX, trainY, testX, testY, final_feature)

    print('Training finished successfully')
    
if __name__ == "__main__":
    tf.app.run()

-----------------------------------------------------------------
## Final features and Data pre-process for prediction
-----------------------------------------------------------------
       FlowDuration  BwdIATTot  BwdPktLenMax  FlowIATStd  BwdPktLenMean  \
7678   4.808377e-06   0.000000      0.000000    0.000000       0.000000   
7355   4.150038e-06   0.000000      0.000000    0.000000       0.000000   
1638   4.050037e-06   0.000000      0.000000    0.000000       0.000000   
5709   4.183371e-06   0.000000      0.000000    0.000000       0.000000   
276    2.347830e-03   0.000000      0.000000    0.004407       0.000000   
...             ...        ...           ...         ...            ...   
3842   7.796737e-05   0.000075      0.076712    0.000093       0.023585   
11563  8.333409e-09   0.000000      0.000000    0.000000       0.000000   
9802   9.260917e-05   0.000089      0.076712    0.000111       0.023585   
882    9.928674e-01   0.983333      0.034932    0.013026       0

/home/jovyan/.local/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [23] are constant.
  UserWarning)
/home/jovyan/.local/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
I0421 09:58:49.229977 140556435072832 estimator.py:209] Using config: {'_model_dir': 'network/', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.tra

INFO:tensorflow:Not using Distribute Coordinator.


I0421 09:58:49.232479 140556435072832 estimator_training.py:186] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

INFO:tensorflow:Loss for final step: 21.61299.


I0421 09:58:52.789323 140556435072832 estimator.py:368] Loss for final step: 21.61299.


({'accuracy': 0.53125, 'accuracy_baseline': 0.53125, 'auc': 0.7431372, 'auc_precision_recall': 0.7068181, 'average_loss': 0.6070869, 'label/mean': 0.46875, 'loss': 19.42678, 'precision': 0.0, 'prediction/mean': 0.4088927, 'recall': 0.0, 'global_step': 1}, [b'network/export/network/1587463132'])
Training finished successfully


SystemExit: 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Update  storageUri in network_kfserving.yaml with pvc-name

In [16]:
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
! sed -i "s/nfs/$pvc/g" network_kfserving.yaml
! cat network_kfserving.yaml

apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "network-model"
  namespace: anonymous
spec:
  default:
    predictor:
      tensorflow:
        storageUri: "pvc://workspace-network/network/export/network"


## Serving Network Traffic Model using kubeflow kfserving

In [17]:
!kubectl apply -f network_kfserving.yaml -n anonymous

inferenceservice.serving.kubeflow.org/network-model created


In [21]:
!kubectl get inferenceservices -n anonymous

NAME            URL                                                                  READY   DEFAULT TRAFFIC   CANARY TRAFFIC   AGE
network-model   http://network-model.anonymous.example.com/v1/models/network-model   True    100                                28s


#### Note:
Wait for inference service READY=\"True\"

## Predict data from serving after setting INGRESS_IP
### Note - Use one of preprocessed row values from Data pre-process for prediction output cell

In [22]:
! curl -v -H "Host: network-model.anonymous.example.com" http://<<INGRESS_IP>>:<<PORT>>/v1/models/network-model:predict -d '{"signature_name":"predict","instances":[{"FlowDuration":[0.45808569] , "BwdPktLenMax":[0.62440011] , "FlowIATStd":[2.35424276], "FwdPSHFlags":[0.45808569] , "BwdPktLenMean":[0.62440011] , "FlowIATMean":[2.35424276] , "BwdIATMean":[0.45808569] , "FwdSegSizeMin":[0.62440011] , "InitBwdWinByts":[2.35424276] , "BwdPktLenMin":[0.62440011]}]}'

*   Trying 10.30.118.172...
* TCP_NODELAY set
* Connected to 10.30.118.172 (10.30.118.172) port 31380 (#0)
> POST /v1/models/network-model:predict HTTP/1.1
> Host: network-model.anonymous.example.com
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Length: 339
> Content-Type: application/x-www-form-urlencoded
> 
* upload completely sent off: 339 out of 339 bytes
< HTTP/1.1 200 OK
< content-length: 318
< content-type: application/json
< date: Tue, 21 Apr 2020 10:05:58 GMT
< x-envoy-upstream-service-time: 9568
< server: istio-envoy
< 
{
    "predictions": [
        {
            "logits": [-1.10299587],
            "class_ids": [0],
            "classes": ["0"],
            "all_class_ids": [0, 1],
            "logistic": [0.249178976],
            "all_classes": ["0", "1"],
            "probabilities": [0.750821054, 0.249178976]
        }
    ]
* Connection #0 to host 10.30.118.172 left intact
}

## Delete kfserving model & Clean up of stored models

In [23]:
!kubectl delete -f network_kfserving.yaml
!rm -rf /mnt/network
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
! sed -i "s/$pvc/nfs/g" network_kfserving.yaml
! cat network_kfserving.yaml

inferenceservice.serving.kubeflow.org "network-model" deleted
apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "network-model"
  namespace: anonymous
spec:
  default:
    predictor:
      tensorflow:
        storageUri: "pvc://nfs/network/export/network"
